In [3]:
import matplotlib
#matplotlib.use('Agg')
import os
import datetime
import time
import copy
import shutil
import sys
sys.path.append('/cnrm/pastel/USERS/lecestres/analyse/')
from function_read import *
from HW_detection import *
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
# from mpl_toolkits.basemap import Basemap, shiftgrid

import calendar
import locale
locale.setlocale( locale.LC_ALL , 'en_US' )
from netCDF4 import num2date, date2num 
import netCDF4
from math import sin, cos, sqrt, atan2, radians
from sklearn.neighbors import DistanceMetric
from math import radians

from joblib import Parallel, delayed
import joblib

In [4]:
### EXPERIENCE NAME
expname = "ocean_reanalysis_GREP"
#expname = "sst_retroprevision_sys7"

### PERCENTILE THRESHOLD
percent_thresh = 95
#percent_thresh = 90

### MINIMAL DURATION OF A HW
duration_min = 5
#duration_min = 3

### REGION OF EXPERIENCE

#reg_name = 'north_pacific'
#reg_name = 'north_atlantic'
#reg_name = 'indian_ocean'
#reg_name = 'austral_ocean'
#reg_name = 'tropical_atlantic'
#reg_name = 'tropical_pacific'
#reg_name = 'mediterranee'
reg_name = 'global'

if reg_name == 'north_pacific':
    lats_bnds = np.array([30,65])
    lons_bnds = np.array([120, -120])
if reg_name == 'north_atlantic':
    lats_bnds = np.array([30,65])
    lons_bnds = np.array([-80, 0])
if reg_name == 'indian_ocean':
    lats_bnds = np.array([-30,30])
    lons_bnds = np.array([45, 110])
if reg_name == 'austral_ocean':
    lats_bnds = np.array([-90,-30])
    lons_bnds = np.array([-180, 180])
if reg_name == 'tropical_atlantic':
    lats_bnds = np.array([-30,30])
    lons_bnds = np.array([-70, 20])
if reg_name == 'tropical_pacific':
    lats_bnds = np.array([-30,30])
    lons_bnds = np.array([120, -70])
if reg_name == 'mediterranee':
    lats_bnds = np.array([30,50])
    lons_bnds = np.array([-5, 40])
if reg_name == 'global':
    lats_bnds = np.array([-90,90])
    lons_bnds = np.array([-180,180])
    
nlat = lats_bnds[1]-lats_bnds[0]
if lons_bnds[1]<lons_bnds[0]:
    nlon = lons_bnds[1]%360-lons_bnds[0]
else:
    nlon = lons_bnds[1]-lons_bnds[0]

### YEARS
if expname == 'ocean_reanalysis_GREP':
    end_year=2016
    start_year=1993
elif expname == 'sst_retroprevision_sys7':
    end_year=2016
    start_year=1993
nyear=end_year-start_year+1

### SEASON
#season = "NDJFMAM"
season = "DJF"
if season == 'NDJFMAM':
    nday = 211
    season_start_day = [11,1] #1stNov
    season_end_day = [5,31] #31stMay
    first_day = 0
elif season == 'DJF':
    nday = 90
    season_start_day = [12,1] #1stDec
    season_end_day = [3,1] #28thFeb
    first_day = 30
ndayseas = nday//duration_min +1

if expname == "ocean_reanalysis_GREP":    
    ### NUMBER OF MEMBS
    first_memb=0
    last_memb=1
    nmemb = last_memb-first_memb
    
    ### CROSS VALIDATION
    cv = True
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'
        
elif expname == "sst_retroprevision_sys7":    
    ### NUMBER OF MEMBS
    first_memb=0
    last_memb=25
    nmemb = last_memb-first_memb    ### NUMBER OF MEMBS
    
    ### CROSS VALIDATION
    cv = True
    if cv:
        cv_str = "CV"
    else:
        cv_str = 'notCV'

### PARAMETERS
# Parameters will be initialized later in the script
#parameters_str = reg_name + "_" + season + "_" + cv_str + '_percent%i'%(percent_thresh) + '_daymin%i'%(duration_min) +  "ref%i-%i"%(start_year, end_year) + '_' + memb_str

### NAME OF THE VARIABLES IN THE NC FILES
#varname = 'HWMI_' + parameters_str
#varname = 'subHW_' + parameters_str

In [5]:
def HWs_detection_function(j, jmemb):
    memb_str = 'memb'+str(jmemb)
    parameters_str = reg_name+"_"+season+"_"+cv_str+'_percent%i'%(percent_thresh)+'_daymin%i'%(duration_min)+"_ref%i-%i_"%(start_year, end_year)+memb_str
    varname = 'subHW_'+parameters_str
    pathHWMI = '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/'+expname+'/'+memb_str+'/'+season+'/'+varname+'/'
    files = glob(pathHWMI + '*.nc')
    files.sort()
    allsubHWslst = []
    for file in files[:]:
        varf=netCDF4.Dataset(file)
        varf.variables[varname]
        vararray, lats_reg, lons_reg = extract_array(varf, varname, ndayseas, np.array(lons_bnds), np.array(lats_bnds),  start_time = 0, level=0)
        subHWs_iyeararray = vararray[:, np.newaxis, :, :]
        maskobs = subHWs_iyeararray.mask
        allsubHWslst.append(subHWs_iyeararray)
    allsubHWs_array = np.ma.array(allsubHWslst)
    #print('maskobs : ', maskobs)
    args = (expname, reg_name, memb_str, season, parameters_str, start_year, lats_reg, lons_reg)
    start_time = time.time()
    print(allsubHWs_array.shape)
    calc_HW_MY(allsubHWs_array, maskobs, lats_reg, lons_reg, args, allowdist=1)
    print('Total time for detection : ', time.time() - start_time)

In [6]:
for (j, jmemb) in enumerate(range(first_memb, last_memb)):
    memb_str = 'memb'+str(jmemb)
    parameters_str = reg_name+"_"+season+"_"+cv_str+'_percent%i'%(percent_thresh)+'_daymin%i'%(duration_min)+"_ref%i-%i_"%(start_year, end_year)+memb_str
    varname = 'subHW_'+parameters_str
    pathHWMI = '/cnrm/pastel/USERS/lecestres/NO_SAVE/data/'+expname+'/'+memb_str+'/'+season+'/'+varname+'/'
    files = glob(pathHWMI + '*.nc')
    files.sort()
    allsubHWslst = []    
    def parallel_years(k_start_year, k_end_year):
        year_range = k_end_year-k_start_year
        k_min = k_start_year-start_year
        k_max = k_min+year_range
        for file in files[k_min:k_max]:
            varf=netCDF4.Dataset(file)
            varf.variables[varname]
            vararray, lats_reg, lons_reg = extract_array(varf, varname, ndayseas, np.array(lons_bnds), np.array(lats_bnds),  start_time = 0, level=0)
            subHWs_iyeararray = vararray[:, np.newaxis, :, :]
            maskobs = subHWs_iyeararray.mask
            allsubHWslst.append(subHWs_iyeararray)
        allsubHWs_array = np.ma.array(allsubHWslst)
        #print('maskobs : ', maskobs)
        args = (expname, reg_name, memb_str, season, parameters_str, k_start_year, lats_reg, lons_reg)
        print(allsubHWs_array.shape)
        calc_HW_MY(allsubHWs_array, maskobs, lats_reg, lons_reg, args, allowdist=1)
        
    CPUs = os.cpu_count()
    years_per_CPU = int(np.ceil(nyear/CPUs))
    k_list = [(start_year + i*(years_per_CPU), min(start_year + (i+1)*years_per_CPU, end_year+1)) for i in range(CPUs)]
    
    start_time = time.time()
    Parallel(n_jobs=min(CPUs,nyear), backend='multiprocessing', verbose = 20)(delayed(parallel_years)(k_start_year, k_end_year) for (k_start_year, k_end_year) in k_list)
    print('Total time for detection : ', time.time() - start_time)

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/nilshempelmann/mambaforge/envs/heatwaves/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/nilshempelmann/mambaforge/envs/heatwaves/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/nilshempelmann/mambaforge/envs/heatwaves/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/nilshempelmann/mambaforge/envs/heatwaves/lib/python3.10/site-packages/joblib/pool.py", line 147, in get
    return recv()
  File "/Users/nilshempelmann/mambaforge/envs/heatwaves/lib/python3.10/multiprocessing/connection.py", line 256, in recv
    return _ForkingPickler.loads(buf.getbuffer())
AttributeError: Can't get attribute 'parallel_years' on <module '__main__' (built-in)>
Process SpawnPoolWork

KeyboardInterrupt: 